In [1]:
# !ls -l
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import io
import math
import os

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer, QuantileTransformer
from sklearn.metrics import accuracy_score, average_precision_score, roc_curve
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

from xgboost import XGBClassifier
from xgboost import plot_tree
# import shap

import seaborn as sns
import pickle

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
with open('parametersFromCV.pickle', 'rb') as handle:
    b = pickle.load(handle)
for k in b.keys():
  print(k)

64pp_all_DA_64pp_A
64pp_all_DA_64pp_B
64pp_5kBTSS_DA_64pp_A
64pp_5kBTSS_DA_64pp_B
64pp_5kATSS_DA_64pp_A
64pp_5kATSS_DA_64pp_B
64pp_5kBTES_DA_64pp_A
64pp_5kBTES_DA_64pp_B
64pp_intergenic_DA_64pp_A
64pp_intergenic_DA_64pp_B
CPD_all_DA_CPD_A
CPD_all_DA_CPD_B
CPD_5kBTSS_DA_CPD_A
CPD_5kBTSS_DA_CPD_B
CPD_5kATSS_DA_CPD_A
CPD_5kATSS_DA_CPD_B
CPD_5kBTES_DA_CPD_A
CPD_5kBTES_DA_CPD_B
CPD_intergenic_DA_CPD_A
CPD_intergenic_DA_CPD_B


In [0]:
dataDir = '/content/drive/My Drive/00Projects/DNA Repair/prettyFigures/melanoma/data'
figDir = '/content/drive/My Drive/00Projects/DNA Repair/prettyFigures/melanoma/figures'
os.chdir(dataDir)

In [3]:
!ls -ltr

total 255614
-rw------- 1 root root 124406512 Apr 30 11:02 64pp_all_DA_normalized.csv
-rw------- 1 root root 123861465 Apr 30 11:03 CPD_all_DA_normalized.csv
-rw------- 1 root root   1549745 Apr 30 11:07 64pp_5kBTSS_DA_normalized.csv
-rw------- 1 root root   1546310 Apr 30 11:08 CPD_5kBTSS_DA_normalized.csv
-rw------- 1 root root   1552196 Apr 30 11:10 64pp_5kATSS_DA_normalized.csv
-rw------- 1 root root   1546500 Apr 30 11:10 CPD_5kATSS_DA_normalized.csv
-rw------- 1 root root   1550419 Apr 30 11:11 64pp_5kBTES_DA_normalized.csv
-rw------- 1 root root   1548445 Apr 30 11:11 CPD_5kBTES_DA_normalized.csv
-rw------- 1 root root   1529388 Apr 30 11:13 64pp_intergenic_DA_normalized.csv
-rw------- 1 root root   1516760 Apr 30 11:13 CPD_intergenic_DA_normalized.csv
-rw------- 1 root root   1138049 Apr 30 16:43 parametersFromCV.pickle


#Functions

## getRepairDF

In [0]:
def getRepairRegionDF(repairType, region, damageSample):
  '''
  repairType: one of ['64pp', 'CPD' or 'Cisplatin']
  region: one of ['all', '5kBTSS', '5kATSS', '5kBTES', 'intergenic']
  damageSample: one of ['DA', 'DB']
  '''
  fname = '.'.join(['_'.join([repairType, region, damageSample, 'normalized']), 'csv'])
  dat = pd.read_csv(fname)
  return dat

## binDF

In [0]:
def binDF(df, numBins):
    binner = KBinsDiscretizer(n_bins=numBins, encode='ordinal', strategy='quantile')
    binner.fit(df)
    binnedDf = binner.transform(df)
    binnedDf = pd.DataFrame(binnedDf)
    binnedDf.columns = df.columns
    return binnedDf

## getTrainTestData

In [0]:
def getTrainTestData(dat, response, chromatin_type):

  '''
  chromtin_type: one of ['All', 'open', 'mid', 'close']
  '''
  
  def binDF(df, numBins):
    binner = KBinsDiscretizer(n_bins=numBins, encode='ordinal', strategy='quantile')
    binner.fit(df)
    binnedDf = binner.transform(df)
    binnedDf = pd.DataFrame(binnedDf)
    binnedDf.columns = df.columns
    return binnedDf


  # if chromatin_type == 'none':    
    # rem0HS = dat[dat.DNAse_I != 0]
    # rpkm = rem0HS.filter(regex='H')
    # X = rpkm.div(rem0HS.DNAse_I, axis=0)    
  # else:
  rpkm = dat.filter(regex='H')
  X = binDF(rpkm, numBins = 4)  #Features
  if chromatin_type != 'All':
    HS = pd.DataFrame(dat['DNAse_I']) # Add HS info 
    binnedHS = binDF(HS, numBins = 3)
    X['binnedHS'] = binnedHS
    X = X.astype({'binnedHS': 'int32'})

  repair = pd.DataFrame(dat[response]) # Bizarize response 
  binnedRep = binDF(repair, numBins = 3)
  X['binnedRep'] = binnedRep
  X = X.astype({'binnedRep': 'int32'})
  X['binnedRep'] = X['binnedRep'].map({0: -1, 1: np.nan, 2:1})
  X = X[X.binnedRep.notnull()]

  y = X['binnedRep']  #Response
  colsToDrop = ['binnedRep']
  x = X.drop(colsToDrop, axis = 1)
  # Put aside test data (contains both open & closed chromatin examples)
  xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2, shuffle = False)

  if chromatin_type == 'All':
    return xTrain, xTest, yTrain, yTest
  else:    
    xTest['binnedRep'] = yTest
    # Test data subset based on open/mid/closed chromatin
    if chromatin_type == 'close':  
      xTest_n = xTest[xTest['binnedHS'] == 0]
    elif chromatin_type == 'mid':
      xTest_n = xTest[xTest['binnedHS'] == 1]
    elif chromatin_type == 'open':
      xTest_n = xTest[xTest['binnedHS'] == 2]

    yTest = xTest_n['binnedRep']  
    colsToDrop = ['binnedHS', 'binnedRep']
    xTest_n = xTest_n.drop(colsToDrop, axis = 1)    
    colsToDrop = ['binnedHS']
    xTrain = xTrain.drop(colsToDrop, axis = 1)
    return xTrain, xTest_n, yTrain, yTest

# getTrainTestData RPKM

In [0]:
def getTrainTestData(dat, response, chromatin_type):

  '''
  chromtin_type: one of ['All', 'open', 'mid', 'close']
  '''
  
  def binDF(df, numBins):
    binner = KBinsDiscretizer(n_bins=numBins, encode='ordinal', strategy='quantile')
    binner.fit(df)
    binnedDf = binner.transform(df)
    binnedDf = pd.DataFrame(binnedDf)
    binnedDf.columns = df.columns
    return binnedDf

  rpkm = dat.filter(regex='H')
  X = rpkm
  #  X = binDF(rpkm, numBins = 4)  #Features
  if chromatin_type != 'All':
    HS = pd.DataFrame(dat['DNAse_I']) # Add HS info 
    binnedHS = binDF(HS, numBins = 3)
    X['binnedHS'] = binnedHS
    X = X.astype({'binnedHS': 'int32'})

  repair = pd.DataFrame(dat[response]) # Bizarize response 
  binnedRep = binDF(repair, numBins = 3)
  X['binnedRep'] = binnedRep
  X = X.astype({'binnedRep': 'int32'})
  X['binnedRep'] = X['binnedRep'].map({0: -1, 1: np.nan, 2:1})
  X = X[X.binnedRep.notnull()]

  y = X['binnedRep']  #Response
  colsToDrop = ['binnedRep']
  x = X.drop(colsToDrop, axis = 1)
  # Put aside test data (contains both open & closed chromatin examples)
  xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2, shuffle = False)

  if chromatin_type == 'All':
    return xTrain, xTest, yTrain, yTest
  else:    
    xTest['binnedRep'] = yTest
    # Test data subset based on open/mid/closed chromatin
    if chromatin_type == 'close':  
      xTest_n = xTest[xTest['binnedHS'] == 0]
    elif chromatin_type == 'mid':
      xTest_n = xTest[xTest['binnedHS'] == 1]
    elif chromatin_type == 'open':
      xTest_n = xTest[xTest['binnedHS'] == 2]

    yTest = xTest_n['binnedRep']  
    colsToDrop = ['binnedHS', 'binnedRep']
    xTest_n = xTest_n.drop(colsToDrop, axis = 1)    
    colsToDrop = ['binnedHS']
    xTrain = xTrain.drop(colsToDrop, axis = 1)
    return xTrain, xTest_n, yTrain, yTest

## plotMaxDepthVsAUC

In [0]:
def plotMaxDepthVsAUC(xTrain, yTrain, best_tree, repairType, region, response):
  max_depths = [1, 2, 3, 4, 5, 6, 7, 8]
  aucs = []
  auc_sds = []
  for max_depth in max_depths: 
      model = XGBClassifier(max_depth=max_depth, n_estimators=best_tree)
      CVaucs = cross_val_score(model, xTrain, yTrain, cv=4, scoring='roc_auc')
      aucs.append(CVaucs.mean())
      auc_sds.append(CVaucs.std())
      
  plt.plot(max_depths, aucs)
  plt.plot(max_depths, np.array(aucs) + np.array(auc_sds), 'b--', )
  plt.plot(max_depths, np.array(aucs) - np.array(auc_sds), 'b--', )
  plt.fill_between(max_depths, np.array(aucs) + np.array(auc_sds), np.array(aucs) - np.array(auc_sds), alpha=0.2, facecolor ='b')
  plt.xticks(max_depths )
  plt.xlabel(max_depths )
  plt.ylabel('CV AUC')
  titName = ' '.join([repairType, region, response, 'Trees:', str(best_tree)])
  plt.title(titName)
  figName = [repairType, region, response, str(best_tree), 'Trees' ,'MaxDepthsVsAUC']
  figName = '_'.join(figName)
  figName = figName + '.jpeg'
  plt.savefig(figName, format='jpeg', dpi=500) 
  plt.show()

## plotNumTreesVsAUC

In [0]:
def plotNumTreesVsAUC(xTrain, yTrain, repairType, response, region):
  trees = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 75, 100, 150, 200]
  aucs = []
  auc_sds = []
  for tree in trees: 
      model = XGBClassifier(n_estimators=tree)
      CVaucs = cross_val_score(model, xTrain, yTrain, cv=4, scoring='roc_auc')
      aucs.append(CVaucs.mean())
      auc_sds.append(CVaucs.std())
      
  plt.plot(trees, aucs)
  plt.plot(trees, np.array(aucs) + np.array(auc_sds), 'b--', )
  plt.plot(trees, np.array(aucs) - np.array(auc_sds), 'b--', )
  plt.fill_between(trees, np.array(aucs) + np.array(auc_sds), np.array(aucs) - np.array(auc_sds), alpha=0.2, facecolor ='b')
  plt.xticks(trees )
  plt.xlabel(trees )
  plt.ylabel('CV AUC')
  titName = ' '.join([repairType, region, response])
  plt.title(titName)
  figName = [repairType, region, response, 'NumTreesVsAUC']
  figName = '_'.join(figName)
  figName = figName + '.jpeg'
  plt.savefig(figName, format='jpeg', dpi=500) 
  plt.show()

## crossValidate

In [0]:
def crossValidate(xTrain, yTrain, 
                  best_tree, best_depth, 
                  repairType, region, damageSample, response):
  
  print(repairType, region, damageSample, response)
  myClassifier = XGBClassifier()

  # CV for hyperparameter tuning
  paramDict = {}
  paramDict["n_estimators"] = [best_tree]
  paramDict["max_depth"] = [best_depth]
  paramDict["min_child_weight"] = [0, 0.5, 1, 2, 3, 4]
  paramDict["subsample"] = [0.2, 0.3, 0.5, 0.7, 0.8]
  paramDict['gamma'] = [0, 1, 2, 3, 4]

  myGridSearch = GridSearchCV(myClassifier, param_grid = paramDict,
                    scoring = "roc_auc", cv = 4)
  myGridSearch.fit(xTrain, yTrain)

  print("Best parameters:")
  print(myGridSearch.best_params_)
  paramToUseDict = myGridSearch.best_params_  
  return paramToUseDict

# Main

In [11]:
# repairTypes = ['64pp', 'CPD', 'Cisplatin']
repairTypes = ['64pp', 'CPD']

regions = ['all', '5kBTSS', '5kATSS', '5kBTES', 'intergenic']
# regions = ['all']

# damageSamples = ['DA', 'DB']
damageSamples = ['DA']

parameterDict = {}

for repairType in repairTypes:
  for region in regions:
    for damageSample in damageSamples:
      dat = getRepairRegionDF(repairType, region, damageSample) #get relevant df
      hists = dat.filter(regex='H')
      dat = dat.loc[(hists!=0).any(1)] # remove rows w/ histones all 0
      
      responses = ['_'.join([repairType, 'A']),
                  '_'.join([repairType, 'B'])] 

      for response in responses:
        # Chromatin type given in the one below does not matter
        xTrain, xTest, yTrain, yTest = getTrainTestData(dat, response, 
                                                        chromatin_type = 'open')
        thisParamDict = crossValidate(xTrain, yTrain, 
                      best_tree = 50,  best_depth=4, 
                      repairType = repairType,
                      region=region, 
                      damageSample=damageSample,
                      response=response)
        thisParamDictName = '_'.join([repairType, region, damageSample, response])
        parameterDict[thisParamDictName] = thisParamDict
        with open('parametersFromCV_zero_rows_removed.pickle', 'wb') as handle:
          pickle.dump(parameterDict, handle)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


64pp all DA 64pp_A
Best parameters:
{'gamma': 2, 'max_depth': 4, 'min_child_weight': 0, 'n_estimators': 50, 'subsample': 0.2}


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


64pp all DA 64pp_B
Best parameters:
{'gamma': 1, 'max_depth': 4, 'min_child_weight': 2, 'n_estimators': 50, 'subsample': 0.3}


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

Best parameters:
{'gamma': 1, 'max_depth': 4, 'min_child_weight': 0.5, 'n_estimators': 50, 'subsample': 0.7}


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPD intergenic DA CPD_A
Best parameters:
{'gamma': 4, 'max_depth': 4, 'min_child_weight': 2, 'n_estimators': 50, 'subsample': 0.8}
CPD intergenic DA CPD_B


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Best parameters:
{'gamma': 4, 'max_depth': 4, 'min_child_weight': 2, 'n_estimators': 50, 'subsample': 0.3}


In [0]:
with open('parametersFromCV.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [0]:
b == parameterDict

False

In [0]:
parameterDict

{'64pp_5kATSS_DA_64pp_A': XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=3,
               learning_rate=0.1, max_delta_step=0, max_depth=4,
               min_child_weight=0.5, missing=None, n_estimators=50, n_jobs=1,
               nthread=None, objective='binary:logistic', random_state=0,
               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
               silent=None, subsample=0.5, verbosity=1),
 '64pp_5kATSS_DA_64pp_B': XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=0,
               learning_rate=0.1, max_delta_step=0, max_depth=4,
               min_child_weight=1, missing=None, n_estimators=50, n_jobs=1,
               nthread=None, objective='binary:logistic', random_state=0,
               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
               silent=None, subsample=0.8, verb

In [0]:
for k in b.keys():
  print(k)

64pp_all_DA_64pp_A
64pp_all_DA_64pp_B
64pp_5kBTSS_DA_64pp_A
64pp_5kBTSS_DA_64pp_B
64pp_5kATSS_DA_64pp_A
64pp_5kATSS_DA_64pp_B
64pp_5kBTES_DA_64pp_A
64pp_5kBTES_DA_64pp_B
64pp_intergenic_DA_64pp_A
64pp_intergenic_DA_64pp_B
CPD_all_DA_CPD_A
CPD_all_DA_CPD_B
CPD_5kBTSS_DA_CPD_A
CPD_5kBTSS_DA_CPD_B
CPD_5kATSS_DA_CPD_A
CPD_5kATSS_DA_CPD_B
CPD_5kBTES_DA_CPD_A
CPD_5kBTES_DA_CPD_B
CPD_intergenic_DA_CPD_A
CPD_intergenic_DA_CPD_B


In [0]:
for j in parameterDict.keys():
  print(j)

64pp_all_DA_64pp_A
64pp_all_DA_64pp_B
64pp_5kBTSS_DA_64pp_A
64pp_5kBTSS_DA_64pp_B
64pp_5kATSS_DA_64pp_A
64pp_5kATSS_DA_64pp_B
64pp_5kBTES_DA_64pp_A
64pp_5kBTES_DA_64pp_B
64pp_intergenic_DA_64pp_A
64pp_intergenic_DA_64pp_B
CPD_all_DA_CPD_A
CPD_all_DA_CPD_B
CPD_5kBTSS_DA_CPD_A
CPD_5kBTSS_DA_CPD_B
CPD_5kATSS_DA_CPD_A
CPD_5kATSS_DA_CPD_B
CPD_5kBTES_DA_CPD_A
CPD_5kBTES_DA_CPD_B
CPD_intergenic_DA_CPD_A
CPD_intergenic_DA_CPD_B


In [0]:
print(b['64pp_all_DA_64pp_A']== parameterDict['64pp_all_DA_64pp_A'])

False
